In [34]:
import os
import importlib
import melee 
import numpy as np
import pandas as pd
import data_processing 
importlib.reload(data_processing) 
from data_processing import process_slp_file, save_processed_data 

In [35]:
os.getcwd()

'/Users/alexoon/Desktop/smash_bot/icl_smash'

In [36]:
# Process a single file
slp_file = "./data/Stream-Game_20220828T223051.slp"
sampled_data, columns = process_slp_file(slp_file)
sampled_data.shape

# Save the results
save_processed_data(sampled_data, columns, "./training_data", "[libmeleeV2]training_ex1.npy", save_csv=True)

# Or use the main function directly
# shape = main(slp_file, "./training_data", "DESIRED_FILENAME")

Saved CSV to: ./training_data/[libmeleeV2]training_ex1.npy_data.csv


In [37]:
columns

['frame',
 'distance',
 'p3_action_frame',
 'p3_controller_c_stick_x',
 'p3_controller_c_stick_y',
 'p3_controller_l_shoulder',
 'p3_controller_r_shoulder',
 'p3_controller_main_stick_x',
 'p3_controller_main_stick_y',
 'p3_button_button_a',
 'p3_button_button_b',
 'p3_button_button_x',
 'p3_button_button_y',
 'p3_button_button_z',
 'p3_button_button_l',
 'p3_button_button_r',
 'p3_button_button_start',
 'p3_button_button_d_up',
 'p3_button_button_d_down',
 'p3_button_button_d_left',
 'p3_button_button_d_right',
 'p3_ecb_bottom_x',
 'p3_ecb_bottom_y',
 'p3_ecb_left_x',
 'p3_ecb_left_y',
 'p3_ecb_right_x',
 'p3_ecb_right_y',
 'p3_ecb_top_x',
 'p3_ecb_top_y',
 'p3_facing',
 'p3_hitlag_left',
 'p3_hitstun_frames_left',
 'p3_iasa',
 'p3_invulnerability_left',
 'p3_invulnerable',
 'p3_jumps_left',
 'p3_moonwalkwarning',
 'p3_nana',
 'p3_off_stage',
 'p3_on_ground',
 'p3_percent',
 'p3_position_x',
 'p3_position_y',
 'p3_shield_strength',
 'p3_speed_air_x_self',
 'p3_speed_ground_x_self',
 '

In [38]:
df = pd.read_csv("/Users/alexoon/Desktop/smash_bot/icl_smash/training_data/[libmeleeV2]training_ex1.npy_data.csv")
df.head() 

,frame,distance,p3_action_frame,p3_controller_c_stick_x,p3_controller_c_stick_y,p3_controller_l_shoulder,p3_controller_r_shoulder,p3_controller_main_stick_x,p3_controller_main_stick_y,p3_button_button_a,...,proj1_owner,proj1_position_x,proj1_position_y,proj1_speed_x,proj1_speed_y,stage,p3_action,p3_character,p4_action,p4_character
0,-123,94.000052,-1,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY,SHEIK,ENTRY,JIGGLYPUFF
1,-108,94.000021,1,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY_START,SHEIK,ENTRY,JIGGLYPUFF
2,-93,94.000026,11,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY_START,SHEIK,ENTRY_START,JIGGLYPUFF
3,-78,94.000010,-1,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY_END,SHEIK,ENTRY_START,JIGGLYPUFF
4,-63,94.000639,-1,0.5,0.5,0.0,0.0,0.5,0.5,1,...,NaN,NaN,NaN,NaN,NaN,DREAMLAND,ENTRY_END,SHEIK,ENTRY_END,JIGGLYPUFF


In [39]:
df['p3_controller_r_shoulder']

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
837    0.0
838    0.0
839    0.0
840    0.0
841    0.0
Name: p3_controller_r_shoulder, Length: 842, dtype: float64